# Linear Regression - Housing data

Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


Check out the data

In [ ]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

USAhousing = pd.concat([ train, test ])

In [ ]:
USAhousing.head()

In [ ]:
USAhousing.describe()

In [ ]:
USAhousing.info()
# sns.heatmap(combined.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    print(USAhousing.isnull().sum())

In [ ]:
# sns.heatmap(USAhousing.isnull(),yticklabels=False,cbar=False,cmap='viridis')

plt.figure(figsize=(20, 20))

sns.heatmap(USAhousing.isnull(),
            square=False,
            annot=False,
            yticklabels=False,
            cbar=False,
            cmap='viridis'            
           )

plt.title('Features with missing values');

Correlation between variables:

In [ ]:
# sns.heatmap(USAhousing.corr())

corr=USAhousing.corr()

plt.figure(figsize=(20, 20))

sns.heatmap(corr, 
            vmax=.8, 
            linewidths=0.01,
            square=True,
            annot=True,
            cmap='YlGnBu',
            linecolor="white")

plt.title('Correlation between features');

Train a linear model

Clean Data

In [ ]:
# Clean Alley column
USAhousing.fillna(value={
    'Alley': 'NA', 
    'Fence': 'NA', 
    'FireplaceQu':'NA', 
    'MiscFeature':'NA',
    'PoolQC': 'NA',
    'BsmtCond': 'NA',
    'BsmtExposure': 'NA',
    'BsmtFinType1': 'NA',
    'BsmtFinType2': 'NA',
    'BsmtFinSF1': 0,
    'BsmtFinSF2': 0,
    'BsmtFullBath': 0,
    'BsmtHalfBath': 0,
    'BsmtUnfSF': 0,
    'TotalBsmtSF': 0,
    'BsmtQual': 'NA',
    'GarageCond': 'NA',
    'GarageFinish': 'NA',
    'GarageQual': 'NA',
    'GarageType': 'NA',
    'GarageYrBlt': USAhousing['YearBuilt'],
    'MasVnrArea': 0,
    'MasVnrType': 'None',
    'Electrical': 'SBrkr',
    'Functional': 'Typ',
    'GarageArea': 0,
    'GarageCars': 0,
    'MSZoning': 'RL',
    'Utilities': 'AllPub',
    'Exterior1st': 'VinylSd',
    'Exterior2nd': 'VinylSd',
    'KitchenQual': 'TA',
    'SaleType': 'WD'    
}, inplace=True)

# drop lotfrontage
USAhousing.drop('LotFrontage', axis=1, inplace=True)

Create dummy variables


In [ ]:
objColumns = [col for col in list(USAhousing.columns) if USAhousing[col].dtypes == object]

dummies = pd.get_dummies(USAhousing, columns=objColumns, drop_first=True)

USAhousing = USAhousing.drop(objColumns, axis=1)

result = pd.concat([USAhousing, dummies], axis=1)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(result[:5])

In [ ]:
train = USAhousing[USAhousing['SalePrice'].notnull()]

test = USAhousing[USAhousing['SalePrice'].isnull()]
test = test.drop('SalePrice', axis=1)

In [ ]:
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']


Train Test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

Create and train the model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_train, y_train)

# Predictions

In [ ]:
predictions = lm.predict(X_test)

In [ ]:
plt.scatter(y_test, predictions)

In [ ]:
sns.distplot((y_test - predictions))

In [ ]:
from sklearn import metrics

In [ ]:
metrics.mean_absolute_error(y_test, predictions)

In [ ]:
metrics.mean_squared_error(y_test, predictions)

In [ ]:
np.sqrt(metrics.mean_squared_error(y_test, predictions))

# Random Forest Regressor model


In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfc = RandomForestRegressor(n_estimators=100)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

In [ ]:
plt.scatter(y_test, rfc_pred)


In [ ]:
sns.distplot((y_test - rfc_pred))

In [ ]:
print 'MAE:  ', metrics.mean_absolute_error(y_test, rfc_pred)
print 'MSE:  ', metrics.mean_squared_error(y_test, rfc_pred)
print 'RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, rfc_pred))

# SVR model

In [ ]:
from sklearn.svm import SVR

In [ ]:
svr_rbf = SVR(kernel='linear', C=1e3)

In [ ]:
svr_rbf.fit(X_train, y_train)
svr_rbf_pred = svr_rbf.predict(X_test)

In [ ]:
plt.scatter(y_test, svr_rbf_pred)

In [ ]:
sns.distplot((y_test - svr_rbf_pred))

In [ ]:
print 'RSME: ', np.sqrt(metrics.mean_squared_error(y_test, svr_rbf_pred)) 